# Analysis and cleaning of evictions data
Notebook to get year over year and within-borough comparisons of eviction counts for 2024. 

In [84]:
import pandas as pd
import numpy as np

In [157]:
df = pd.read_csv("nyc_evictions.csv")

In [96]:
df.head()

,Court Index Number,Docket Number,Eviction Address,Eviction Apartment Number,Executed Date,Marshal First Name,Marshal Last Name,Residential/Commercial,BOROUGH,Eviction Postcode,Ejectment,Eviction/Legal Possession,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,12882/18,480750,2907 KINGSBRIDGE TER RACE,42A,01/11/2019,Danny,Weinheim,Residential,BRONX,10463,Not an Ejectment,Possession,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55795/16,1407,717 EAST 228TH ST.,3D,11/16/2017,Salavatore,Giglio,Residential,BRONX,10466,Not an Ejectment,Possession,40.889691,-73.858535,12.0,12.0,406.0,2063558.0,2.048420e+09,Williamsbridge-Olinville
2,72819/18,91735,40-36 67TH STREET,NaN,05/29/2019,Henry,Daley,Residential,QUEENS,11377,Not an Ejectment,Possession,40.745572,-73.898012,2.0,26.0,263.0,4030222.0,4.012980e+09,Woodside
3,52723/17,76440,601 EAST 156TH ST,9D,01/18/2018,Henry,Daley,Residential,BRONX,10455,Not an Ejectment,Possession,40.819056,-73.909938,1.0,17.0,75.0,2117693.0,2.026188e+09,Melrose South-Mott Haven North
4,66394/19,101938,732 FLATBUSH AVENUE GROUND FLOOR COMMERCIAL SPACE,NaN,01/20/2023,Henry,Daley,Commercial,BROOKLYN,11226,Not an Ejectment,Possession,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# some date parsing and column creation
df['month_year'] = pd.to_datetime(df['Executed Date']).dt.strftime('%m %Y')
df['month'] = pd.to_datetime(df['Executed Date']).dt.strftime('%m')
df['month_name'] = pd.to_datetime(df['Executed Date']).dt.strftime('%b')

df['year'] = pd.to_datetime(df['Executed Date']).dt.strftime('%Y').astype(int)

## Basic YoY
Get year over year comparisons and copy to clipboard for Datawrapper

In [161]:
df[df['year'] < 2025].groupby(["year"])['Court Index Number'].count().to_clipboard()

## YoY cumulative sums
I knew 'good cause' laws were passed in April, so I wanted to see if the spike was only due to early months or if evictions increased consistently (they did).

In [127]:
# filter for applicable years
cumsum_df = df[(df['year'] > 2020) & (df['year'] < 2025)]

In [128]:
# calculate rolling sum
cumsum_df = (cumsum_df
             .groupby(["year", "month", "month_name"])['Court Index Number']
             .count()
             .groupby(level=0)
             .cumsum())

In [111]:
# change to df to pivot for dartawrapper
cumsum_df = pd.DataFrame(cumsum_df)

In [114]:
# pivot months into columns
pivot_df = cumsum_df.reset_index().pivot(index="year", columns="month_name", values="Court Index Number")
pivot_df.head()

month_name,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
year,,,,,,,,,,,,
2021,11,124,268,7,4,78,26,8,17,196,154,135
2022,888,2585,5067,306,118,2087,1676,595,1270,4600,3807,3140
2023,3383,8589,13447,1682,965,7110,5883,2604,4492,12542,11273,9835
2024,5379,11106,16753,2894,1563,9601,8087,4089,6827,15529,14271,12563


In [116]:
pivot_df.to_clipboard()

## Borough YoY
I also want to see if any boroughs stood out in the increase or if it was consistent city-wide

In [130]:
# intial count
year_count_df = df.groupby(["BOROUGH", "year"])['Court Index Number'].count()


In [135]:
# pivot with years to columns
boro_year_df = (
    year_count_df
    .reset_index()
    .pivot(index="BOROUGH", columns="year", values="Court Index Number")
)

In [142]:
boro_year_df = boro_year_df.reset_index()

In [146]:
print(boro_year_df.columns)


Index(['BOROUGH', 2021, 2022, 2023, 2024], dtype='object', name='year')


In [148]:
# calculate percent diff
boro_year_df['perc_diff'] = (boro_year_df[2024] - boro_year_df[2023]) / boro_year_df[2023]

In [151]:
# calculate raw count diff
boro_year_df['diff'] = (boro_year_df[2024] - boro_year_df[2023])

In [152]:
boro_year_df

year,BOROUGH,2021,2022,2023,2024,perc_diff,diff
0,BRONX,29,1175,4320,4693,0.086343,373
1,BROOKLYN,100,1871,3927,4452,0.133690,525
2,MANHATTAN,68,934,2614,3140,0.201224,526
3,QUEENS,36,813,2022,3764,0.861523,1742
4,STATEN ISLAND,35,274,564,704,0.248227,140


In [153]:
boro_year_df.to_clipboard()